# 22.目标导向的强化学习（goal-oriented reinforcement learning，GoRL）
> 之前学习的如 **PPO、SAC** 等经典的深度强化学习算法，智能体的行为通常是根据固定的奖励函数来学习的，只关注单一目标。算法训练完后，在使用它们时只能完成一个特定的任务。
> 而在许多实际任务场景中，目标是复杂的，涉及到 **多个子目标或必须在长期内达成的目标**。之前传统的强化学习算法往往不容易训练出有效的策略。

> **目标导向的强化学习（GoRL）** 的概念并非源自单一的原始论文，而是逐渐在多个领域中演化和扩展的一个 **思想**：
> - 让智能体在前往 **最终目标** 的训练过程中将所经历的 **失败经验** 也作为一种 **目标** 进行学习，使智能体理解 **错误目标** 是如何达成的，最终学习一种 **策略**：在应对 **不同的目标（goal）** 依然奏效。以此来解决较为复杂的 **复合任务**。

> 本节介绍该思想下的一种经典算法 [事后经验回放（hindsight experience replay，HER）](https://arxiv.org/abs/1707.01495)：主要是为了解决 **稀疏奖励**

## 22.1 场景定义